In [ ]:
import time
start_time = time.time()
total_time = (4 * 60 + 45) * 60

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# https://www.kaggle.com/competitions/ai-mathematical-olympiad-progress-prize-2/discussion/560682#3113134
os.environ["TRITON_PTXAS_PATH"] = "/usr/local/cuda/bin/ptxas"
os.environ["VLLM_USE_V1"] = "1"

In [ ]:
import pandas as pd
import polars as pl

import kaggle_evaluation.aimo_2_inference_server

In [ ]:
from transformers import set_seed
set_seed(42)

In [ ]:
def get_environment() -> str:
    if os.getenv("KAGGLE_KERNEL_RUN_TYPE") != None:
        return "kaggle"
    elif os.getenv("COLAB_RELEASE_TAG") != None:
        return "colab"
    else:
        return "local"


ENV = get_environment()
if ENV == "kaggle":
    from jeffdshen.aimo2.predict import MetaLLM
    from jeffdshen.aimo2.systems import WeightedEnsemble
    from jeffdshen.aimo2.config import (
        get_validation_data,
        load_llm,
        MODELS,
        SYSTEM_PARAMS,
    )
elif ENV == "local":
    from .predict import MetaLLM
    from .systems import WeightedEnsemble
    from .config import get_validation_data, load_llm, MODELS, SYSTEM_PARAMS
elif ENV == "colab":
    raise NotImplementedError("Not yet implemented")
else:
    raise ValueError("Unknown environment")

In [ ]:
data = get_validation_data()

In [ ]:
llm = load_llm(MODELS["deepseek-r1-distill-qwen-32b-awq"])

In [ ]:
SYSTEMS = [
    "r1_v1a",
    "r1_v1b",
    "r1_v1c",
    "r1_v2a",
    "r1_v2b",
    "r1_v2c",
]

In [ ]:
with open("request_log.jsonl", "w") as f:
    system = WeightedEnsemble(
        system_params=[SYSTEM_PARAMS[x] for x in SYSTEMS],
        question_log=f,
        correct_answers=data.correct_answers,
        start_time=start_time,
        full_timeout=total_time,
    )
    meta_llm = MetaLLM(
        llm=llm,
        system=system,
    )

    inference_server = kaggle_evaluation.aimo_2_inference_server.AIMO2InferenceServer(meta_llm.predict)

    if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
        inference_server.serve()
    else:
        inference_server.run_local_gateway(
            (
                '/kaggle/input/ai-mathematical-olympiad-progress-prize-2/test.csv',
            )
        )
